In [1]:
from beta import Beta

In [5]:
beta = Beta('PYPL', start='2019-1-1', end='2020-8-1')

PYPL/SPY::
Correlation: 0.5734573548346732
Covariance: 285.1883456625234
Variance: 477.74349159741473

BETA:  0.5969486778541948


### Porfolio Creation

In [60]:
portfolio = ['TSLA', 'MCD', 'WMT', 'COST', 'ARCT', 'CRSP', 'TWST', 
             'EDIT', 'VCYT', 'DEO', 'STZ', 'LMND', 'ETSY', 'WORK', 
             'ZM', 'NET', 'OSTK', 'DOCU', 'SAP', 'SQ', 'PYPL', 
             'NVDA', 'AMD', 'INTL', 'NVTA', 'PRLB', 'ROKU', 'Z', 
             'SNAP', 'AAPL', 'PLTM', 'SILV', 'GLD']

### Process 
  1. Retrieve the daily charts for each ticker
  2. Calculate Sharpe ratio relative to S&P500 Xyear
  3. Minimize for the -Sharpe ratio, such that sum of all allocations = 1.0

In [12]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import sys

In [13]:
md = yf.Ticker('SPY')
df = md.history(period='max', start='2019-1-1', end='2020-8-1')
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,238.95,244.03,238.92,243.03,126925200,0.0,0
2019-01-03,241.13,241.46,236.70,237.23,144140700,0.0,0
2019-01-04,240.51,245.87,240.10,245.17,142628800,0.0,0
2019-01-07,245.46,248.63,244.49,247.11,103139100,0.0,0
2019-01-08,249.48,249.95,246.74,249.43,102512600,0.0,0
...,...,...,...,...,...,...,...
2020-07-27,321.63,323.41,320.77,323.22,48293000,0.0,0
2020-07-28,322.43,323.64,320.85,321.17,57495000,0.0,0
2020-07-29,322.12,325.73,322.08,325.12,48454200,0.0,0


In [14]:
df['Open']

Date
2019-01-02    238.95
2019-01-03    241.13
2019-01-04    240.51
2019-01-07    245.46
2019-01-08    249.48
               ...  
2020-07-27    321.63
2020-07-28    322.43
2020-07-29    322.12
2020-07-30    321.90
2020-07-31    325.90
Name: Open, Length: 399, dtype: float64

In [61]:
# for each ticker in portfolio 
# retrieve data from the last x time 
# store data into a 3d dataframe where
# x = ticker, y = day, z = O,H,L,C,V
class Portfolio():
    benchmark = None
    portfolio = []
    
    daily_open = pd.DataFrame()
    daily_high = pd.DataFrame()
    daily_low = pd.DataFrame()
    daily_close = pd.DataFrame()
    daily_volume = pd.DataFrame()
    
    def __init__(self, portfolio=None, benchmark='SPY', start='2019-1-1', end='2020-8-1'): #todo, default end today
        self.start = start
        self.end = end
        self.benchmark = benchmark
        data = self.pull_market_data(self.benchmark)
        self.initialize_charts(data)
        if portfolio != None:
            self.add_securities(portfolio)
        
    def pull_market_data(self, ticker):
        df = pd.DataFrame()
#         try:
        #print(type(ticker), ticker)
        md = yf.Ticker(ticker)
        df = md.history(period='max', start=self.start, end=self.end)
        return df
#         except Exception as e:
#             print(str(e))
        
        #return -1
    
    def initialize_charts(self, df):
        ''' Takes the benchmark df and initializes a dataframe for each element 
            of a daily candle: Open, High, Low, Close, and Volume'''
        self.daily_open = df['Open'].rename(columns={"Open": self.benchmark}).rename(self.benchmark)
        self.daily_high = df['High'].rename(columns={"High": self.benchmark}).rename(self.benchmark)
        self.daily_low = df['Low'].rename(columns={"Low": self.benchmark}).rename(self.benchmark)
        self.daily_close = df['Close'].rename(columns={"Close": self.benchmark}).rename(self.benchmark)
        self.daily_volume = df['Volume'].rename(columns={"Volume": self.benchmark}).rename(self.benchmark)
        return 0
    
    def initialize_indicators(self, df):
        ''' Initializes a dataframe for each indicator based on the data in the 
            daily candles for the benchmark'''
        return 0
        
    def add_securities(self, securities):
        ''' Takes a list, securities, and adds each element in the list to our portfolio'''
        securities = [ s for s in securities if s not in self.portfolio]
        securities = [ s for s in securities if self.validate_ticker(s)]
        self.portfolio.extend( securities )
        
        for asset in self.portfolio:
            df = self.pull_market_data(asset)
            #TODO: modularize code below with self.initialize_charts()
            tmp_open = df['Open'].rename(columns={"Open": asset}).rename(asset)
            tmp_high = df['High'].rename(columns={"High": asset}).rename(asset)
            tmp_low = df['Low'].rename(columns={"Low": asset}).rename(asset)
            tmp_close = df['Close'].rename(columns={"Close": asset}).rename(asset)
            tmp_volume = df['Volume'].rename(columns={"Volume": asset}).rename(asset)
            
            #print(self.daily_open.name, self.daily_open[:1], '\n\n', tmp_open.name,tmp_open[:1])
            self.daily_open = pd.merge(self.daily_open, tmp_open, on='Date', how='left').bfill()
            self.daily_high = pd.merge(self.daily_high, tmp_high, on='Date', how='left').bfill()
            self.daily_low = pd.merge(self.daily_low, tmp_low, on='Date', how='left').bfill()
            self.daily_close = pd.merge(self.daily_close, tmp_close, on='Date', how='left').bfill()
            self.daily_volume = pd.merge(self.daily_volume, tmp_volume, on='Date', how='left').bfill()
            
            
        
    def remove_securities(self, securities):
        ''' Takes a list, securities, and removes each element from our portfolio'''
        self.portfolio = [ s for s in self.portfolio if s not in securities]
    
    # TODO: verify that ticker legit 
    def validate_ticker(self, ticker):
        return True

In [62]:
p = Portfolio(portfolio=portfolio)
#p.daily_close

In [64]:
exp_df = p.daily_close.iloc[-1]/p.daily_close.iloc[0]-1
exp_df

SPY      0.343538
TSLA     3.613569
MCD      0.144776
WMT      0.426839
COST     0.615934
ARCT    10.136461
CRSP     1.861065
TWST     1.291087
EDIT     0.257926
VCYT     1.972500
DEO      0.089698
STZ      0.112707
LMND    -0.161792
ETSY     1.518723
WORK    -0.234852
ZM       3.095323
NET      1.312222
OSTK     4.349611
DOCU     4.311857
SAP      0.650804
SQ       1.270105
PYPL     1.286531
NVDA     2.132350
AMD      3.112055
INTL     0.426087
NVTA     1.581786
PRLB     0.076441
ROKU     3.762915
Z        1.253377
SNAP     2.872193
AAPL     1.745914
PLTM     0.131479
SILV     2.526502
GLD      0.528311
dtype: float64